In [86]:
# Det kan ofta vara användbart att sammanställa hela kolumner.
# Att ta ut ett resultat från många kolumner kallas att aggregera.

# Det kan vara en summa
weather["Rainfall"].sum() # Det här blir då hur mycket det regnat totalt under hela datasettet.

# Ett genomsnitt
weather["MinTemp"].mean()

# Ett max
weather["MaxTemp"].max()

#Eller ett min
weather["MinTemp"].min()

# Eller hur många icke NaN det finns
weather["MinTemp"].count()

# Eller hur många unika värden det finns.
weather["Location"].nunique()



49

In [87]:
# Ofta är det dock mer intressant att inte bara ta utt ett värde per kolumn.
# Utan man kanske är intresserad av den varmaste temperaturen uppmätt för varje plats?
weather.groupby("Location")["MaxTemp"].max()

# Det du grupperar på blir också index. Om du inte vill ha det så kan resetta det.
weather.groupby("Location")["MaxTemp"].max().reset_index()

# Du kan också gruppera efter fler kolumner. 
#Låt oss säga att jag skulle vilja ha den genomsnittliga nederbörden för varje plats.
weather["Date"] = pd.to_datetime(weather["Date"])
weather["Month"] = weather["Date"].dt.month
monthly_rainfall = weather.groupby(["Location", "Month"])["Rainfall"].mean().reset_index()

# Du kan sedan sortera värdena.
monthly_rainfall.sort_values(["Location", "Month"])


,Location,Month,Rainfall
0,Adelaide,1,0.769118
1,Adelaide,2,1.005357
2,Adelaide,3,1.187407
3,Adelaide,4,1.416102
4,Adelaide,5,2.216988
...,...,...,...
583,Woomera,8,0.364516
584,Woomera,9,0.419167
585,Woomera,10,0.348387
586,Woomera,11,0.441250


In [88]:
orders = pd.read_csv("orders.csv")
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")

# Du kan merga två df med pd.merge. Den letar då automatiskt efter kolumner med samma namn och mergar dem.
pd.merge(orders, products)

# Du kan också göra såhär:
orders.merge(products)

# Ofta anses det dock ett anti-pattern / bad practice att nyttja s.k NATURAL JOINS.
# Det kan skapa oväntade problem om datan ändras.
# Dessutom hade nog kolumnen egentligen hetat id i products, inte product_id.
orders.rename(columns={"order_id": "id"}, inplace=True)
products.rename(columns={"product_id": "id"}, inplace=True)
customers.rename(columns={"customer_id": "id"}, inplace=True)

# Nu ger pd.merge upphov till märkliga beteenden.
pd.merge(orders, products)

# Bättre är att specifiera vad du joinar på.
pd.merge(orders, products, left_on="product_id", right_on="id", suffixes=["_order", "_product"])

# Likt SQL kan du också olika typer av joins.
pd.merge(customers, orders, left_on="id", right_on="customer_id", how="left")

,id_x,customer_name,address,phone_number,id_y,customer_id,product_id,quantity,timestamp
0,1,John Smith,123 Main St.,212-123-4567,1.0,1.0,3.0,1.0,2017-01-01
1,1,John Smith,123 Main St.,212-123-4567,6.0,1.0,4.0,2.0,2017-03-01
2,1,John Smith,123 Main St.,212-123-4567,7.0,1.0,1.0,1.0,2017-02-02
3,1,John Smith,123 Main St.,212-123-4567,8.0,1.0,4.0,1.0,2017-02-02
4,2,Jane Doe,456 Park Ave.,949-867-5309,NaN,NaN,NaN,NaN,NaN
5,3,Joe Schmo,798 Broadway,112-358-1321,2.0,3.0,2.0,3.0,2017-01-01
6,3,Joe Schmo,798 Broadway,112-358-1321,3.0,3.0,1.0,1.0,2017-01-01
7,3,Joe Schmo,798 Broadway,112-358-1321,4.0,3.0,2.0,2.0,2017-02-01
8,3,Joe Schmo,798 Broadway,112-358-1321,5.0,3.0,3.0,3.0,2017-02-01


In [89]:
# Du kan också göra motsvarande UNIONS.
customers2 = pd.read_csv("customers2.csv")
pd.concat([customers, customers2])

# Har du mismatchade kolumner kommer resultatet inte att bli lika bra.

,id,customer_name,address,phone_number
0,1,John Smith,123 Main St.,212-123-4567
1,2,Jane Doe,456 Park Ave.,949-867-5309
2,3,Joe Schmo,798 Broadway,112-358-1321
0,4,Anna Andersson,Strandvägen 10,123-123-123


In [90]:
# Exempeluppgift: Ta reda på vilken den varmaste platsen i Australien varje enskilt datum.
# Utgå från att MaxTemp är representativt.

hottest_location_idx = weather.groupby("Date")["MaxTemp"].idxmax()

# Man kan egentligen göra såhär, men vi ska joina för uppgiftensskull.
weather.loc[hottest_location_idx]

hottest_locations = pd.merge(hottest_location_idx, weather, left_on="MaxTemp", right_index=True)
hottest_locations = hottest_locations[["Location", "MaxTemp_y"]]
hottest_locations.rename(columns={"MaxTemp_y": "MaxTemp"})

,Location,MaxTemp
Date,,
2007-11-01,Canberra,24.3
2007-11-02,Canberra,26.9
2007-11-03,Canberra,23.4
2007-11-04,Canberra,15.5
2007-11-05,Canberra,16.1
...,...,...
2017-06-21,Darwin,32.6
2017-06-22,Darwin,32.8
2017-06-23,Darwin,31.8


In [91]:
# För riktigt stora dataset används oftast inte csv.
# Man använder sig ofta av parquet, feather eller HDF.
# Dessa format komprimerar data och bevarar dessutom metadata som datatyper hos kolumner.

weather.to_parquet("weather.parquet", index=False)
weather_parquet = pd.read_parquet("weather.parquet")
weather_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           145460 non-null  datetime64[ns]
 1   Location       145460 non-null  object        
 2   MinTemp        143975 non-null  float64       
 3   MaxTemp        144199 non-null  float64       
 4   Rainfall       142199 non-null  float64       
 5   Evaporation    82670 non-null   float64       
 6   Sunshine       75625 non-null   float64       
 7   WindGustDir    135134 non-null  object        
 8   WindGustSpeed  135197 non-null  float64       
 9   WindDir9am     134894 non-null  object        
 10  WindDir3pm     141232 non-null  object        
 11  WindSpeed9am   143693 non-null  float64       
 12  WindSpeed3pm   142398 non-null  float64       
 13  Humidity9am    142806 non-null  float64       
 14  Humidity3pm    140953 non-null  float64       
 15  

In [92]:
# Rätt datatyper viktigt för att kunna tolka din data rätt.
# Det sparar också plats.
weather["Location"] = weather["Location"].astype("category")
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           145460 non-null  datetime64[ns]
 1   Location       145460 non-null  category      
 2   MinTemp        143975 non-null  float64       
 3   MaxTemp        144199 non-null  float64       
 4   Rainfall       142199 non-null  float64       
 5   Evaporation    82670 non-null   float64       
 6   Sunshine       75625 non-null   float64       
 7   WindGustDir    135134 non-null  object        
 8   WindGustSpeed  135197 non-null  float64       
 9   WindDir9am     134894 non-null  object        
 10  WindDir3pm     141232 non-null  object        
 11  WindSpeed9am   143693 non-null  float64       
 12  WindSpeed3pm   142398 non-null  float64       
 13  Humidity9am    142806 non-null  float64       
 14  Humidity3pm    140953 non-null  float64       
 15  

In [93]:
# Ponera att vi vill klassifiera varje rad som antingen kall, varm eller mittemellan, kanske för en algoritm.
# Jag vill kategorisera enligt:
# -15 grader = kall
# 15 - 25 grader = mittemellan
# 25+ = varm
# Det finns många sätt att göra detta på. Här kommer de från sämsta till bästa.



In [94]:
# En dålig approach är att loopa igenom raderna.
weather = pd.read_csv("weatherAUS.csv")
def classify_temperature(temp):
    if temp < 15:
        return "kall"
    elif temp < 25:
        return "mittemellan"
    return "varm"

classification = []
for i in range(len(weather)):
    temp = classify_temperature(weather.iloc[i]["MinTemp"])
    classification.append(temp)
weather["Classification"] = classification
weather

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Classification
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No,kall
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No,kall
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No,kall
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No,kall
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,mittemellan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No,kall
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No,kall
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No,kall
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,...,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No,kall


In [100]:
# Något bättre (men fortfarande dåligt är att loopa igenom med .iterrows())
weather = pd.read_csv("weatherAUS.csv")
classification = []
for index, row in weather.iterrows():
    temp = classify_temperature(row["MinTemp"])
    classification.append(temp)
weather["Classification"] = classification
weather

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Classification
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No,kall
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No,kall
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No,kall
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No,kall
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,mittemellan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No,kall
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No,kall
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No,kall
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,...,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No,kall


In [ ]:
weather = pd.read_csv("weatherAUS.csv")

In [101]:
%%timeit
# En bra metod är att använda inbygga funktioner som kör över hela dataframen.
# Här använder vi våra egna funktioner skrivna i Python.

weather["Classification"] = weather["MinTemp"].apply(lambda x: classify_temperature(x))

30.9 ms ± 1.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [102]:
weather = pd.read_csv("weatherAUS.csv")

In [103]:
%%timeit

# Allra bästa är pandas inbyggda funktioner. De är skrivna i C.

bins = [-float("inf"), 15, 25, float("inf")]
labels = ["kall", "mittemellan", "varm"]

weather["Classification"] = pd.cut(
    weather["MinTemp"], bins=bins, labels=labels
)
weather

5.33 ms ± 499 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
